In [19]:
import pandas as pd
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
if ROOT_DIR not in sys.path:
    sys.path.insert(0, ROOT_DIR)

from packages.utils import cloud_cnxn
c_con, c_eng = cloud_cnxn()

In [20]:
# Ensure all schema have same number of columns for game_details
from packages.utils import seasons, season_conversion
seasons = seasons()

all_dfs = []
for season in seasons:
    schema = season_conversion(season)
    temp_df = pd.read_sql_query(f"""select * from "{schema}".game_details where home_pts is not null""", con=c_eng)
    temp_df["schema"] = schema
    print(schema,len(temp_df))
    all_dfs.append(temp_df)
    del temp_df

df = pd.concat(all_dfs, ignore_index=True)
df.set_index("game_id", inplace=True)
df.shape


202122 1323
202223 1320
202324 1318
202425 1321
202526 586


(5868, 29)

In [21]:
# Evaluate data
records = len(df)

def std_officials(officials):
    return "_".join(sorted(officials.split("_")))

df["officials"] = df.officials.apply(lambda x: std_officials(x))

for col in df.columns:
    print(col)
    nulls = df[col].isnull().sum()
    null_pct = round(100 * nulls/records)
    print(f"""Nulls: {nulls}, {null_pct}%  Type:{df[col].dtype}""")
    vc = df[col].value_counts()
    print(vc)
    print()

game_type
Nulls: 0, 0%  Type:object
game_type
Regular Season          5373
Playoffs                 337
Emirates NBA Cup         133
PlayIn                    24
NBA Mexico City Game       1
Name: count, dtype: int64

home_id
Nulls: 0, 0%  Type:object
home_id
1610612738    223
1610612748    209
1610612743    208
1610612744    207
1610612750    207
1610612760    206
1610612754    205
1610612752    205
1610612739    203
1610612742    202
1610612755    198
1610612749    198
1610612756    197
1610612747    196
1610612763    196
1610612740    195
1610612746    192
1610612758    190
1610612761    190
1610612753    190
1610612751    189
1610612741    188
1610612737    188
1610612762    188
1610612765    185
1610612757    184
1610612766    183
1610612764    183
1610612759    182
1610612745    181
Name: count, dtype: int64

away_id
Nulls: 0, 0%  Type:object
away_id
1610612738    218
1610612748    213
1610612743    210
1610612744    210
1610612742    208
1610612750    206
1610612752    204
16106

In [22]:
# Officals combinations of three happen surpisingly infrequently, with 4 occurrences being the maximum
# Start with what they have direct control over: fouls
with open(f"{ROOT_DIR}/scripts/officials.sql", "r") as f:
    base_sql = f.read()

all_dfs = []
for season in seasons:
    schema = season_conversion(season)
    sql = base_sql.replace('"202526"', f'"{schema}"')
    fouls = pd.read_sql_query(sql, con=c_eng)
    print(schema,len(fouls))
    fouls["season"] = season
    all_dfs.append(fouls)
    del fouls

df_fouls = pd.concat(all_dfs, ignore_index=True)
df_fouls.shape


202122 1323
202223 1320
202324 1318
202425 1321
202526 586


(5868, 14)

In [23]:
df_fouls.columns

Index(['game_id', 'home_id', 'away_id', 'officials', 'duration', 'home_fouls',
       'away_fouls', 'fouls_diff', 'fouls_total', 'home_fta', 'away_fta',
       'fta_diff', 'fta_total', 'season'],
      dtype='object')

In [24]:
# Establish baselines
# Use PRE-EXPLODED data
home_pf_base = df_fouls.groupby("home_id")["home_fouls"].mean()  # Average fouls for home teams
away_pf_base = df_fouls.groupby("away_id")["away_fouls"].mean()  # Average fouls for away teams

home_ft_base = df_fouls.groupby("home_id")["home_fta"].mean()  # Average free-throw attempts for home teams
away_ft_base = df_fouls.groupby("away_id")["away_fta"].mean()  # Average free-throw attempts for away teams

# Pursue duration later
#duration_base = (df_fouls.groupby("game_id")["duration"].mean())  # Average game duration


In [25]:
# Officals combinations of three happen surpisingly infrequently, with 4 occurrences being the maximum
# Start with what they have direct control over: fouls
# Some games have four officials, so it's not necessarily df_fouls X 3!!!
df_refs = df_fouls.copy()
df_refs["officials"] = df_refs["officials"].str.split("_")  # Converts string into a list of officials
df_refs = df_refs.explode("officials")                      # Rows are duplicated for each official
df_refs.shape

(17804, 14)

In [26]:
df_refs["expected_fouls"] = df_refs["home_id"].map(home_pf_base) + df_refs["away_id"].map(away_pf_base)
df_refs["fouls_residual"] = df_refs["home_fouls"] + df_refs["away_fouls"] - df_refs["expected_fouls"]

df_refs["expected_fta"] = df_refs["home_id"].map(home_ft_base) + df_refs["away_id"].map(away_ft_base)
df_refs["fta_residual"] = df_refs["home_fta"] + df_refs["away_fta"] - df_refs["expected_fta"]

df_refs.head()

,game_id,home_id,away_id,officials,duration,home_fouls,away_fouls,fouls_diff,fouls_total,home_fta,away_fta,fta_diff,fta_total,season,expected_fouls,fouls_residual,expected_fta,fta_residual
0,0022101137,1610612754,1610612743,Tony Brothers,134,21,20,1,41,23,14,9,37,2021-22,39.291754,1.708246,43.007201,-6.007201
0,0022101137,1610612754,1610612743,Brian Forte,134,21,20,1,41,23,14,9,37,2021-22,39.291754,1.708246,43.007201,-6.007201
0,0022101137,1610612754,1610612743,Jonathan Sterling,134,21,20,1,41,23,14,9,37,2021-22,39.291754,1.708246,43.007201,-6.007201
1,0022100663,1610612742,1610612760,James Williams,133,19,23,-4,42,28,24,4,52,2021-22,39.325433,2.674567,45.824232,6.175768
1,0022100663,1610612742,1610612760,Mitchell Ervin,133,19,23,-4,42,28,24,4,52,2021-22,39.325433,2.674567,45.824232,6.175768
